In [ ]:
#MONDAY 30TH SEPTEMBER
####KSRTC-KERALA              1111

In [26]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def initialize_driver():
    driver = webdriver.Chrome()  #Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    
    try:
        for page in range(1, 3):  #change the range as necessary for pagination
            load_page(driver,"https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile") 

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
                    )
                
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df1 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df1.to_csv('ksrtc_bus_details.csv', index=False)

#loading
df1 = pd.read_csv('ksrtc_bus_details.csv')


In [27]:
df1

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:00,08h 20m,20:20,3.6,INR 627,14 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1210,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,08h 20m,21:15,3.8,INR 627,19 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 50m,23:05,4.3,INR 627,34 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 00m,01:00,3.9,INR 627,4 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,08h 20m,23:45,2.7,INR 627,37 Seats available
...,...,...,...,...,...,...,...,...,...,...
887,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),18:15,07h 10m,01:25,1.0,INR 381,0
888,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Semi Sleeper (2+2),17:30,07h 30m,01:00,3.7,INR 381,0
889,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,05h 30m,04:30,3.7,INR 381,0
890,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Sonia Travels,A/C Semi Sleeper (2+2),18:00,08h 20m,02:20,3.4,INR 381,0


In [ ]:
#####KTCL###
#KADAMBA      22222

In [24]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    
    try:
        for page in range(1, 5):  # Change the range as necessary for pagination
            load_page(driver,"https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
                    )
                
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df2 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df2.to_csv('ktcl_bus_details.csv', index=False)

#loading
df2 = pd.read_csv('ktcl_bus_details.csv')


In [25]:
df2

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.7,INR 1000,2 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.7,INR 1000,11 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.6,INR 1000,17 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,11h 05m,08:05,4.4,INR 1000,32 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.3,INR 1000,22 Seats available
...,...,...,...,...,...,...,...,...,...,...
307,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,11h 30m,20:00,3.4,INR 100,0
308,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Rajat Travels,A/C Seater / Sleeper (2+1),06:30,04h 00m,10:30,3.1,INR 5000,29 Seats available
309,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Yes Express Tours and Travels,Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1),23:00,16h 30m,15:30,3.1,INR 5000,0
310,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,1.2,INR 5000,0


In [ ]:
###rsrtc###
#ANDHRA      33333

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    

    try:
        for page in range(1, 6):  # Change the range as necessary for pagination
            load_page(driver,"https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
                    )
                
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df3 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df3.to_csv('rsrtc_bus_details.csv', index=False)

#loading
df3 = pd.read_csv('rsrtc_bus_details.csv')


In [2]:
df3

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 190115,Express Non AC Seater 2+3,12:00,07h 30m,19:30,5.0,INR 294,42 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 169076,Express Non AC Seater 2+3,12:10,11h 50m,00:00,4.2,INR 294,42 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 184043,Express Non AC Seater 2+3,12:45,06h 15m,19:00,3.1,INR 294,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 183203,Express Non AC Seater 2+3,13:30,06h 00m,19:30,3.6,INR 294,25 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 183141,Express Non AC Seater 2+3,14:35,06h 25m,21:00,1.2,INR 294,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
766,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...,Rinku Travels,A/C Seater / Sleeper (2+1),21:15,05h 40m,02:55,2.7,INR 269,25 Seats available
767,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...,Rinku Travels,NON AC Seater / Sleeper 2+1,21:30,04h 16m,01:46,1.0,INR 269,0
768,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...,SHAGUN CARGO & TRAVELS,NON A/C Sleeper (2+1),20:30,06h 00m,02:30,2.1,INR 269,0
769,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...,Mansoori tour and travels,NON A/C Sleeper (2+1),21:00,04h 45m,01:45,1.9,INR 269,0


In [ ]:
###HRTC###    444

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    

    try:
        for page in range(1, 5):  # Change the range as necessary for pagination

            load_page(driver, "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df4 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df4.to_csv('hrtc_bus_details.csv', index=False)

#loading
df4 = pd.read_csv('hrtc_bus_details.csv')


In [21]:
df4

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 69,Himsuta AC Seater Volvo/Scania 2+2,00:40,08h 50m,09:30,4.7,INR 912,26 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,09h 25m,16:10,4.5,INR 912,42 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 592,A/C Executive (2+3),08:05,10h 05m,18:10,2.7,INR 912,20 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,09h 50m,18:40,2.6,INR 912,35 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,09h 05m,18:30,4.6,INR 912,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
476,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...,City Land Travels,Volvo A/C Semi Sleeper (2+2),20:30,05h 50m,02:20,3.8,INR 459,0
477,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...,Ram Dalal Holidays,Volvo 9600 Multi Axle Semi-Sleeper (2+2),20:38,05h 12m,01:50,5.0,INR 459,0
478,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...,City Land Travels,Volvo A/C Semi Sleeper (2+2),20:40,05h 40m,02:20,3.4,INR 459,0
479,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),20:55,05h 40m,02:35,4.5,INR 459,0


In [ ]:
###JKSRTC###
#JAMMU-KASHMIR    5555

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/jksrtc")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()


    try:
        for page in range(1,6):  # Change the range as necessary for pagination

            load_page(driver, "https://www.redbus.in/online-booking/jksrtc")

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
				)
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df5 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df5.to_csv('jksrtc_bus_details.csv', index=False)

#loading

df5 = pd.read_csv('jksrtc_bus_details.csv')


In [19]:
df5

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,Jamna Travels-Jammu,Non AC Seater (2+2),19:00,11h 45m,06:45,2.1,600,30 Seats available
1,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,New Pal Travels,NON A/C Seater / Sleeper (2+2),19:00,10h 00m,05:00,1.8,600,39 Seats available
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,4.4,INR 2682,28 Seats available
3,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,18h 55m,16:30,4.4,INR 2682,26 Seats available
4,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,2.1,719,34 Seats available
...,...,...,...,...,...,...,...,...,...,...
91,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),04:00,08h 00m,12:00,2.4,INR 400,0
92,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),05:40,07h 50m,13:30,2.4,INR 400,0
93,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),07:00,08h 00m,15:00,2.4,INR 400,0
94,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),06:00,08h 30m,14:30,2.4,INR 400,0


In [3]:
###upsrtc###
    #6666

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
   

    try:
        for page in range(1,6):  # Change the range as necessary for pagination

            load_page(driver,"https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile" )

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") )
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df6 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df6.to_csv('upsrtc_bus_details.csv', index=False)

#loading

df6 = pd.read_csv('upsrtc_bus_details.csv')


In [5]:
df6

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HPR0005,Ordinary Non AC Seater 2+3,11:40,06h 20m,18:00,3.3,INR 444,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0127,Shatabdi AC Seater 2+2,11:49,05h 00m,16:49,4.3,INR 444,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HPR0006,Ordinary Non AC Seater 2+3,12:00,06h 30m,18:30,3.3,INR 444,52 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - GRH0127,Ordinary Non AC Seater 2+3,12:00,06h 30m,18:30,3.3,INR 444,52 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0155,Ordinary Non AC Seater 2+3,12:00,05h 30m,17:30,3.3,INR 444,36 Seats available
...,...,...,...,...,...,...,...,...,...,...
1674,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0146,Ordinary Non AC Seater 2+3,18:52,04h 47m,23:39,3.3,INR 320,0
1675,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - BWR0001,Ordinary Non AC Seater 2+3,20:55,05h 55m,02:50,3.3,INR 320,0
1676,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - SKD0223,Ordinary Non AC Seater 2+3,20:55,04h 25m,01:20,3.3,INR 320,0
1677,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - UNO0024,Ordinary Non AC Seater 2+3,21:35,05h 55m,03:30,3.3,INR 320,0


In [ ]:
###ASTC###
#ASSAM     7777

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
   

    try:
       for page in range(1,6):  # Change the range as necessary for pagination

          load_page(driver, "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

          if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']") 
                ))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

          all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

          for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df7 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df7.to_csv('astc_bus_details.csv', index=False)

#loading
df7 = pd.read_csv('astc_bus_details.csv')

In [14]:
df7

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,04h 00m,09:00,3.7,INR 270,25 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 135927,Volvo AC Seater 2+2,07:10,04h 30m,11:40,4.3,INR 270,28 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 156885,Bharat Benz A/C Seater (2+2),07:35,04h 25m,12:00,4.6,INR 270,0
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 173420,Bharat Benz A/C Seater (2+2),10:35,04h 40m,15:15,3.6,INR 270,0
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,04h 30m,18:30,4.3,INR 270,0
...,...,...,...,...,...,...,...,...,...,...
193,Tezpur to Dibrugarh,https://www.redbus.in/bus-tickets/tezpur-to-di...,J.S. Travels,Non AC Seater (2+1),20:30,07h 30m,04:00,2.3,540,50 Seats available
194,Tezpur to Dibrugarh,https://www.redbus.in/bus-tickets/tezpur-to-di...,MOON TRAVELS,AC Seater (2+2),07:15,08h 05m,15:20,2.8,540,35 Seats available
195,Tezpur to Dibrugarh,https://www.redbus.in/bus-tickets/tezpur-to-di...,SANJEEBANI TRAVELS,Non A/C Seater / Sleeper (2+1),21:30,06h 30m,04:00,2.7,540,0
196,Tezpur to Dibrugarh,https://www.redbus.in/bus-tickets/tezpur-to-di...,Neha Travels,NON A/C Seater (2+2),21:50,08h 40m,06:30,2.1,540,0


In [ ]:
#######  8888
##BSRTC###

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    

    try:
        for page in range(1,5):  # Change the range as necessary for pagination
            load_page(driver, "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']")
                ))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df8 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df8.to_csv('bsrtc_bus_details.csv', index=False)

#loading
df8 = pd.read_csv('bsrtc_bus_details.csv')


In [7]:
df8

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,16:30,16h 00m,08:30,3.2,1020,25 Seats available
1,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,19:10,16h 49m,11:59,3.9,1020,35 Seats available
2,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,ROYAL TRAVELS AND CARGO,A/C Seater / Sleeper (2+2),17:30,15h 00m,08:30,2.2,1020,33 Seats available
3,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,Singh Express Bus Service,A/C Seater / Sleeper (2+2),15:00,19h 00m,10:00,1.0,1020,20 Seats available
4,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...,Rajdhani Express,A/C Seater / Sleeper (2+2),15:00,16h 00m,07:00,1.0,1020,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
341,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Panwar tour and Travels,A/C Seater / Sleeper (2+1),15:40,10h 20m,02:00,1.6,680,34 Seats available
342,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,08h 00m,03:00,1.5,680,58 Seats available
343,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+1),17:00,08h 00m,01:00,1.3,680,47 Seats available
344,Muzaffarpur (Bihar) to Bihar Sharif,https://www.redbus.in/bus-tickets/muzaffarpur-...,Bihar state road transport corporation (BSRTC)...,AC Seater (2+2),18:00,05h 30m,23:30,3.8,INR 275,0


In [ ]:
###tsrtc###  99999

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/north-bengal-state-transport-corporation")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    

    try:
        for page in range(1,5):  # Change the range as necessary for pagination
            load_page(driver,"https://www.redbus.in/online-booking/north-bengal-state-transport-corporation") 

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']")
                ))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df9 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df9.to_csv('nbstc_bus_details.csv', index=False)

#loading
df9 = pd.read_csv('nbstc_bus_details.csv')


In [8]:
df9

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Express Line,Volvo 9600 A/C Seater/Sleeper (2+1),20:00,12h 00m,08:00,4.4,INR 1199,27 Seats available
1,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Poulami Paribahan,Volvo Multi-Axle A/C seater/sleeper (2+1),20:15,11h 15m,07:30,4.1,INR 1199,30 Seats available
2,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Lokenath Bus Service,Volvo Multi Axle B11R AC Seater\Sleeper (2+1),21:00,11h 30m,08:30,4.1,INR 1199,20 Seats available
3,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Modern Travels,Bharat Benz A/C Sleeper (2+1),20:05,11h 25m,07:30,4.8,INR 1199,11 Seats available
4,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Shyamoli Paribahan Pvt Ltd,Volvo 9600 Multi-Axle A/C Sleeper (2+1),20:30,11h 35m,08:05,4.4,INR 1199,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
96,Kolkata to Gangarampur,https://www.redbus.in/bus-tickets/kolkata-to-g...,Express Line,Volvo 9600 A/C Seater/Sleeper (2+1),20:40,09h 05m,05:45,4.5,INR 335,19 Seats available
97,Kolkata to Gangarampur,https://www.redbus.in/bus-tickets/kolkata-to-g...,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),20:00,09h 45m,05:45,3.8,INR 335,19 Seats available
98,Kolkata to Gangarampur,https://www.redbus.in/bus-tickets/kolkata-to-g...,Rubi Paribahan,Volvo 9600 A/C Seater/Sleeper (2+1),20:25,06h 50m,03:15,3.9,INR 335,17 Seats available
99,Kolkata to Gangarampur,https://www.redbus.in/bus-tickets/kolkata-to-g...,Swarnali Paribahan,NON A/C Seater Push Back (2+2),20:20,10h 40m,07:00,2.2,INR 335,5 Seats available


In [ ]:
###pepsu##
        #   10 10

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def initialize_driver():
    driver = webdriver.Chrome()  # Make sure to import webdriver
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
    time.sleep(5)  # Allow the page to load completely

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        view_buses_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(5)

        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Locate bus details
        bus_name_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        arrival_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        fare_element = driver.find_element(By.XPATH, "//div[@class='fare d-block']")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
        
            
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Arrival_Time": arrival_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": fare_element.text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)

        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []


def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    

    try:
        for page in range(1,5):  # Change the range as necessary for pagination
            load_page(driver,("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"))

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,f"//div[contains(@class,'DC_117_pageTabs')][text()='{page}']")
                ))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    
    finally:
        driver.quit()  # Ensure the driver is closed after scraping

    return all_bus_details

# Main execution
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df10 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
#df10.to_csv('pepsu_bus_details.csv', index=False)

# Loading
#df10 = pd.read_csv('pepsu_bus_details.csv')



In [18]:
df10.to_csv('pepsu_bus_details.csv', index=False)


In [19]:
df10=pd.read_csv('pepsu_bus_details.csv')

In [20]:
df10

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66689,Volvo AC Seater (2+2),15:26,04h 59m,20:25,3.8,INR 711,24 Seats available
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67140,AC Seater Hvac 2+2,16:00,05h 30m,21:30,2.7,INR 711,42 Seats available
2,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:30,05h 00m,22:30,4.4,INR 711,42 Seats available
3,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:31,05h 29m,04:00,3.4,INR 711,51 Seats available
4,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),22:00,09h 00m,07:00,4.0,INR 711,51 Seats available
...,...,...,...,...,...,...,...,...,...,...
756,Amritsar to Delhi Airport,https://www.redbus.in/bus-tickets/amritsar-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),21:55,13h 00m,10:55,4.5,INR 1392,51 Seats available
757,Amritsar to Delhi Airport,https://www.redbus.in/bus-tickets/amritsar-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),20:00,11h 00m,07:00,3.0,INR 1392,44 Seats available
758,Amritsar to Delhi Airport,https://www.redbus.in/bus-tickets/amritsar-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),18:00,11h 00m,05:00,2.8,INR 1392,28 Seats available
759,Amritsar to Delhi Airport,https://www.redbus.in/bus-tickets/amritsar-to-...,Golden Temple Volvo Bus Service,A/C Seater / Sleeper (2+2),22:45,08h 55m,07:40,2.8,INR 1392,44 Seats available


In [23]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into a csv format

data = pd.read_csv('astc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



198

In [24]:
import pandas as pd
from sqlalchemy import create_engine



#  connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"



# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv format

data = pd.read_csv('bsrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



346

In [25]:
import pandas as pd
from sqlalchemy import create_engine




# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"



# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# give data into a csv format

data = pd.read_csv('hrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



481

In [26]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv format

data = pd.read_csv('jksrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



96

In [27]:
import pandas as pd
from sqlalchemy import create_engine



#  connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv format

data = pd.read_csv('ksrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



892

In [28]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

#giving the data into csv file

data = pd.read_csv('ktcl_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



312

In [29]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"


# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv file
data = pd.read_csv('nbstc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



101

In [30]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv file

data = pd.read_csv('pepsu_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



761

In [31]:
import pandas as pd
from sqlalchemy import create_engine



# connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv file
data = pd.read_csv('rsrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



771

In [32]:
import pandas as pd
from sqlalchemy import create_engine



# Replace with your connection details
host="localhost"
port="5432"
database="homedb"
user="postgres"
password="mukisam290"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"


# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# giving the data into csv file

data = pd.read_csv('upsrtc_bus_details.csv')

# Table name where you want to push the data
table_name = "bus_details"

# Push the DataFrame to the PostgreSQL table
data.to_sql(table_name, engine,if_exists='append', index=False)  #['fail', 'replace', 'append']



679